In [1]:
import * as dfd from "danfojs-node"
import * as tf from "@tensorflow/tfjs";
const filename = "https://raw.githubusercontent.com/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/master/data/palmer_penguins_openclassrooms.csv"

const data = await dfd.readCSV(filename)

// Explorons rapidement les données avec
console.log(data.head().columns)
console.log(data.describe().toString())




Platform node has already been set. Overwriting the platform with node.
cpu backend was already registered. Reusing existing backend factory.
The kernel '_FusedMatMul' for backend 'cpu' is already registered
The kernel 'Abs' for backend 'cpu' is already registered
The kernel 'Acos' for backend 'cpu' is already registered
The kernel 'Acosh' for backend 'cpu' is already registered
The kernel 'Add' for backend 'cpu' is already registered
The kernel 'AddN' for backend 'cpu' is already registered
The kernel 'All' for backend 'cpu' is already registered
The kernel 'Any' for backend 'cpu' is already registered
The kernel 'ArgMax' for backend 'cpu' is already registered
The kernel 'ArgMin' for backend 'cpu' is already registered
The kernel 'Asin' for backend 'cpu' is already registered
The kernel 'Asinh' for backend 'cpu' is already registered
The kernel 'Atan' for backend 'cpu' is already registered
The kernel 'Atan2' for backend 'cpu' is already registered
The kernel 'Atanh' for backend 'cpu

[
  "species",
  "island",
  "bill_length_mm",
  "bill_depth_mm",
  "flipper_length_mm",
  "body_mass_g",
  "sex",
  "year"
]
╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ bill_length_mm    │ bill_depth_mm     │ flipper_length_…  │ body_mass_g       │ year              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 333               │ 333               │ 333               │ 333               │ 333               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 43.992792792792…  │ 17.164864864864…  │ 200.96696696696…  │ 4207.0570570570…  │ 2008.0420420420…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ std        │ 5.46866834264756  │ 1.9692354633198…  │ 14.01576528828

In [2]:
//normalized
const X = data["bill_length_mm"].values.map(value => (value - Math.min(...data["bill_length_mm"].values)) / (Math.max(...data["bill_length_mm"].values) - Math.min(...data["bill_length_mm"].values)));


const y = data["body_mass_g"].values.map(value => (value - Math.min(...data["body_mass_g"].values)) / (Math.max(...data["body_mass_g"].values) - Math.min(...data["body_mass_g"].values)));
const y2 = data["flipper_length_mm"].values.map(value => (value - Math.min(...data["flipper_length_mm"].values)) / (Math.max(...data["flipper_length_mm"].values) - Math.min(...data["flipper_length_mm"].values)));
const y4 = data["bill_depth_mm"].values.map(value => (value - Math.min(...data["bill_depth_mm"].values)) / (Math.max(...data["bill_depth_mm"].values) - Math.min(...data["bill_depth_mm"].values)));

// Convert data to tensors
const xTensor = tf.tensor2d(X, [X.length, 1]);

//body_mass_g
const yTensor = tf.tensor2d(y, [y.length, 1]);

// Create a simple model
let reg = tf.sequential();
reg.add(tf.layers.dense({ units: 1, inputShape: [1] }));
// Compile the model
reg.compile({
  loss: "meanSquaredError",
  optimizer: "sgd",
});



// Custom function to calculate R²
function calculateR2(yTrue, yPred) {
  const yTrueMean = yTrue.mean().dataSync()[0];
  const ssTotal = yTrue.sub(yTrueMean).square().sum().dataSync()[0];
  const ssResidual = yTrue.sub(yPred).square().sum().dataSync()[0];

  return 1 - (ssResidual / ssTotal);
}
// Train the model
(async () => {
  console.log("body_mass_g");

  await reg.fit(xTensor, yTensor, {
    epochs: 500,
    // callbacks: {
    //   onEpochEnd: (epoch, logs) =>
    //     console.log("Epoch", epoch, "loss", logs.loss),
    // },
  });

  // Use the model to make predictions
  const prediction = reg.predict(xTensor);
  //prediction.print();

  // Calculate R²
  const r2 = calculateR2(yTensor, prediction);
  console.log("body_mass_g R²:", r2);

  // Evaluate the model
  const evalResult = reg.evaluate(xTensor, yTensor);
  const testLoss = evalResult.dataSync()[0];
  console.log("body_mass_g Test Loss:", testLoss);



})();

//////////

//flipper_length_mm
const yTensor2 = tf.tensor2d(y2, [y2.length, 1]);

// Create a simple model
let reg2 = tf.sequential();
reg2.add(tf.layers.dense({ units: 1, inputShape: [1] }));
// Compile the model
reg2.compile({
  loss: "meanSquaredError",
  optimizer: "sgd"
});

// Train the model
(async () => {
  console.log("flipper_length_mm");

  await reg2.fit(xTensor, yTensor2, {
    epochs: 500,
    // callbacks: {
    //   onEpochEnd: (epoch, logs) =>
    //     console.log("Epoch", epoch, "loss", logs.loss),
    // },
  });

    // Use the model to make predictions
    const prediction = reg2.predict(xTensor);
    //prediction.print();
   
   
    // Calculate R²
    const r2 = calculateR2(yTensor2, prediction);
    console.log("flipper_length_mm R²:", r2);

  // Evaluate the model
  const evalResult = reg2.evaluate(xTensor, yTensor);
  const testLoss = evalResult.dataSync()[0];
  console.log("flipper_length_mm Test Loss:", testLoss);
})();

//////////

//bill_depth_mm
const yTensor4 = tf.tensor2d(y4, [y4.length, 1]);

// Create a simple model
let reg4 = tf.sequential();
reg4.add(tf.layers.dense({ units: 1, inputShape: [1] }));
// Compile the model
reg4.compile({
  loss: "meanSquaredError",
  optimizer: "sgd"
});

// Train the model
(async () => {
  console.log("bill_depth_mm");

  await reg4.fit(xTensor, yTensor4, {
    epochs: 500,
    // callbacks: {
    //   onEpochEnd: (epoch, logs) =>
    //     console.log("Epoch", epoch, "loss", logs.loss),
    // },
  });

    // Use the model to make predictions
    const prediction = reg4.predict(xTensor);
    //prediction.print();
   
    // Calculate R²
    const r2 = calculateR2(yTensor4, prediction);
    console.log("bill_depth_mm R²:", r2);

  // Evaluate the model
  const evalResult = reg4.evaluate(xTensor, yTensor4);
  const testLoss = evalResult.dataSync()[0];
  console.log("bill_depth_mm Test Loss:", testLoss);
})();



Hi, looks like you are running TensorFlow.js in Node.js. To speed things up dramatically, install our node backend, visit https://github.com/tensorflow/tfjs-node for more details. 


body_mass_g
flipper_length_mm
bill_depth_mm


Promise { <pending> }

In [3]:
// Select the columns to be normalized
const columnsToNormalize = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"];
const dataToNormalize = data.loc({ columns: columnsToNormalize });

// Apply MinMaxScaler
// Function to perform Min-Max scaling
function minMaxScaler(df) {
    let scaledData = {};
    for (let column of df.columns) {
      const colData = df[column].values;
      const minVal = Math.min(...colData);
      const maxVal = Math.max(...colData);
      scaledData[column] = colData.map(value => (value - minVal) / (maxVal - minVal));
    }
    return new dfd.DataFrame(scaledData);
}

// Convert scaled data back to a DataFrame
const X = minMaxScaler(dataToNormalize);
X.print();

//TODO 

/*

reg.fit(X, y)
print("score",reg.score(X, y))

y_pred = reg.predict(X)


from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

print("mean_squared_error",mean_squared_error(y, y_pred))
print("mean_absolute_error",mean_absolute_error(y, y_pred))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y, y_pred))

*/

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ bill_length_mm    │ bill_depth_mm     │ flipper_length_…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0.3199999999999…  │ 0.4166666666666…  │ 0.2542372881355…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 0.1599999999999…  │ 0.5833333333333…  │ 0.1694915254237…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 2          │ 0.5236363636363…  │ 0.5714285714285…  │ 0.3389830508474…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 3          │ 0.8072727272727…  │ 0.3095238095238…  │ 1                 ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 4          │ 0.5163636363636…  │ 0.3214285714285…  │ 0.7288135593220…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 5          │ 0.15636363

body_mass_g R²: 0.34629470392449213
body_mass_g Test Loss: 0.0326058454811573
flipper_length_mm R²: 0.4265208312209122
flipper_length_mm Test Loss: 0.038300395011901855
bill_depth_mm R²: 0.050559692989443406
bill_depth_mm Test Loss: 0.05202334001660347
